# Add annotation from micrant to other data file

In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
import scipy
import skimage

import os.path as op
import sys
path_to_script = "."
pth = op.join(path_to_script, "../../micrant/")
sys.path.insert(0, pth)
# pth = op.join(path_to_script, "../../scaffan/")
# sys.path.insert(0, pth)

import pandas as pd
import micrant

import micrant.micrant_app
import micrant.image_sort_tools as imst

sns.set_style("ticks")
from pathlib import Path
output_path_images = Path("../docs/")

# df = pd.read_excel("C:/Users/Jirik/data/medical/processed/SA_experiments.xlsx")


Populating the interactive namespace from numpy and matplotlib
add path C:\Users\Jirik\Downloads\openslide-win64-20171122\bin


# Read the data without annotation

The rows are not sorted and they contain each annotation more then onces.

In [2]:

# df = pd.read_excel("C:/Users/Jirik/data/medical/processed/scaffan2019/metacentrum/SA_experiments.xlsx")
df = pd.read_excel(Path("../../scaffan/experiments/SA_experiments.xlsx"))
df

,Annotation Color,Annotation Details,Annotation ID,Annotation Title,Area,Area unit,Branch number,Central Vein Dice,Central Vein Jaccard,Central vein area,...,Processing;Skeleton Analysis;Working Resolution,Processing;Texture Analysis;GLCM Levels,Processing;Texture Analysis;Tile Size,Processing;Texture Analysis;Tile Spacing,Processing;Texture Analysis;Working Resolution,Scaffan Version,Skeleton length,platform.node,platform.processor,platform.system
0,#00FF00,PFCVI=2 SNI=2 SWP=1 SQZ=0.4 CVM=0 IFE=0,6,NaN,0.777046,mm,22489.0,NaN,NaN,0.056655,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,45.58554,krux2.ncbr.muni.cz,NaN,Linux
1,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.8,6,NaN,1.959020,mm,8871.0,NaN,NaN,0.085619,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,18.66592,konos2.fav.zcu.cz,NaN,Linux
2,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,1,NaN,1.562744,mm,NaN,0.108730,0.057491,0.137554,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux
3,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,2,NaN,1.031514,mm,NaN,0.072651,0.037695,0.148508,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux
4,#00FF00,PFCVI=1 SNI=0 SWP=0 SQZ=0.1,6,NaN,1.184912,mm,NaN,0.035911,0.018284,0.148555,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=1.5,9,NaN,0.920082,mm,14435.0,NaN,NaN,0.074718,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,28.39382,elmo5-14.hw.elixir-czech.cz,NaN,Linux
316,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,10,NaN,0.817524,mm,15528.0,NaN,NaN,0.027523,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,29.15094,elmo5-14.hw.elixir-czech.cz,NaN,Linux
317,#0000FF,IFE=0 CVM=0 SQZ=0.1 SNI=2.0,11,NaN,1.190702,mm,15148.0,NaN,NaN,0.119581,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,26.50921,elmo5-14.hw.elixir-czech.cz,NaN,Linux
318,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,12,NaN,1.728910,mm,35651.0,NaN,NaN,0.096388,...,9.100000e-07,64,64,32,4.000000e-07,0.14.0,65.94952,elmo5-14.hw.elixir-czech.cz,NaN,Linux


## Make sure there are the right columns

Columns  `File Name` and `Annotation ID` are required.

In [3]:
df.keys()

Index(['Annotation Color', 'Annotation Details', 'Annotation ID',
       'Annotation Title', 'Area', 'Area unit', 'Branch number',
       'Central Vein Dice', 'Central Vein Jaccard', 'Central vein area',
       'Datetime', 'Dead ends number', 'Experiment Datetime',
       'Experiment Title', 'File Name', 'File Path', 'GLCM Correlation',
       'GLCM Correlation p10', 'GLCM Correlation p25', 'GLCM Correlation p50',
       'GLCM Correlation p75', 'GLCM Correlation p90', 'GLCM Correlation var',
       'GLCM Energy', 'GLCM Energy p10', 'GLCM Energy p25', 'GLCM Energy p50',
       'GLCM Energy p75', 'GLCM Energy p90', 'GLCM Energy var',
       'GLCM Homogenity', 'GLCM Homogenity p10', 'GLCM Homogenity p25',
       'GLCM Homogenity p50', 'GLCM Homogenity p75', 'GLCM Homogenity p90',
       'GLCM Homogenity var', 'Input;Data Info', 'Input;File Path', 'Job ID',
       'Lobulus Border Dice', 'Lobulus Border Jaccard',
       'Lobulus Boundary Compactness', 'Lobulus Equivalent Diameter',
       '

# Read MicrAnt output

There are two options
* Use common spreadsheet file. It is usualy named `micrant_data.xlsx`
* Use spreadsheet with new parameter values usualy named `new_parameter_values.xlsx`

## From common spreadsheet file

In [4]:
# Set parameter name
colname = "HOM"

In [5]:
# mapp = micrant.micrant_app.MicrAnt()
# xfn = mapp.parameters.param("Output", "Common Spreadsheet File").value()
# print(xfn)
xfn = Path("../experiments/micrant_data_paper.xlsx")
df_micrant = pd.read_excel(xfn)
df_micrant #[-5:]

,Annotation Color,Annotation Details,Annotation ID,Annotation Method,Annotation Title,Compared Annotation File Path,Compared Annotation ID,Compared Annotation Parameter Value,Datetime,File Name,File Path,Former Annotation Parameter Value,HOM,MicrAnt Version,Output Directory Path,SNI,platform.node,platform.processor,platform.system
0,#0000FF,SNI=2 IFE=0 SQZ=0.6,0,nothing,NaN,NaN,NaN,NaN,2019-11-20 14:31:15,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,NaN,0.0.0,C:/Users/Jirik/data/SA_20191120_143047,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
1,#0000FF,SNI=2 IFE=0 SQZ=0.4,1,nothing,NaN,NaN,NaN,NaN,2019-11-20 14:31:15,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,NaN,0.0.0,C:/Users/Jirik/data/SA_20191120_143047,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
2,#0000FF,SNI=2 IFE=0 SQZ=0.5,2,nothing,ID=2,NaN,NaN,NaN,2019-11-20 14:31:15,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,NaN,0.0.0,C:/Users/Jirik/data/SA_20191120_143047,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
3,#0000FF,SNI=2 IFE=0 SQZ=0.4,3,nothing,NaN,NaN,NaN,NaN,2019-11-20 14:31:15,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,NaN,0.0.0,C:/Users/Jirik/data/SA_20191120_143047,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
4,#0000FF,SNI=2 IFE=0 SQZ=0.2,5,nothing,NaN,NaN,NaN,NaN,2019-11-20 14:31:15,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,NaN,0.0.0,C:/Users/Jirik/data/SA_20191120_143047,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,NaN,NaN,14,set,NaN,NaN,NaN,NaN,2020-04-11 10:51:15,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,1.00,0.0.3,NaN,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
2909,NaN,NaN,15,set,NaN,NaN,NaN,NaN,2020-04-11 10:51:18,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,1.00,0.0.3,NaN,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
2910,NaN,NaN,15,set,NaN,NaN,NaN,NaN,2020-04-11 10:51:20,PIG-008_P008 LL-P_HE_parenchyme perif..ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,1.00,0.0.3,NaN,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows
2911,NaN,NaN,16,set,NaN,NaN,NaN,NaN,2020-04-11 10:51:25,PIG-008_P008 LL-P_HE_parenchyme perif..ndpi,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,NaN,0.75,0.0.3,NaN,NaN,genomnote4,"Intel64 Family 6 Model 142 Stepping 10, Genuin...",Windows


### Add the column

The last column is `HOM` accoarding to the micrant annotation.

In [6]:
df_new_0 = imst.add_parameter_column(df, df_micrant, colname)
df_new_0

2020-04-11 22:52:32.899 | DEBUG    | micrant.image_sort_tools:get_new_parameter_table:37 - Creating column HOM from Annotation Details


,Annotation Color,Annotation Details,Annotation ID,Annotation Title,Area,Area unit,Branch number,Central Vein Dice,Central Vein Jaccard,Central vein area,...,Processing;Texture Analysis;GLCM Levels,Processing;Texture Analysis;Tile Size,Processing;Texture Analysis;Tile Spacing,Processing;Texture Analysis;Working Resolution,Scaffan Version,Skeleton length,platform.node,platform.processor,platform.system,HOM
0,#00FF00,PFCVI=2 SNI=2 SWP=1 SQZ=0.4 CVM=0 IFE=0,6,NaN,0.777046,mm,22489.0,NaN,NaN,0.056655,...,64,64,32,4.000000e-07,0.14.0,45.58554,krux2.ncbr.muni.cz,NaN,Linux,0.75
1,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.8,6,NaN,1.959020,mm,8871.0,NaN,NaN,0.085619,...,64,64,32,4.000000e-07,0.14.0,18.66592,konos2.fav.zcu.cz,NaN,Linux,0.00
2,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,1,NaN,1.562744,mm,NaN,0.108730,0.057491,0.137554,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
3,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,2,NaN,1.031514,mm,NaN,0.072651,0.037695,0.148508,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
4,#00FF00,PFCVI=1 SNI=0 SWP=0 SQZ=0.1,6,NaN,1.184912,mm,NaN,0.035911,0.018284,0.148555,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=1.5,9,NaN,0.920082,mm,14435.0,NaN,NaN,0.074718,...,64,64,32,4.000000e-07,0.14.0,28.39382,elmo5-14.hw.elixir-czech.cz,NaN,Linux,0.25
316,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,10,NaN,0.817524,mm,15528.0,NaN,NaN,0.027523,...,64,64,32,4.000000e-07,0.14.0,29.15094,elmo5-14.hw.elixir-czech.cz,NaN,Linux,0.75
317,#0000FF,IFE=0 CVM=0 SQZ=0.1 SNI=2.0,11,NaN,1.190702,mm,15148.0,NaN,NaN,0.119581,...,64,64,32,4.000000e-07,0.14.0,26.50921,elmo5-14.hw.elixir-czech.cz,NaN,Linux,1.00
318,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,12,NaN,1.728910,mm,35651.0,NaN,NaN,0.096388,...,64,64,32,4.000000e-07,0.14.0,65.94952,elmo5-14.hw.elixir-czech.cz,NaN,Linux,1.00


## From the new parameter spreadsheet

This file is simplier. It contain Column with new value of parameter. In our case the name of parameter is `HOM`

In [7]:
# np.unique(df["Scaffan Version"])
xfn = Path("../experiments/new_parameter_values_HOM.xlsx")
df_micrant_parameter = pd.read_excel(xfn)
df_micrant_parameter

,Unnamed: 0,Annotation ID,File Name,HOM recent,HOM mean,HOM var,HOM count,HOM last,HOM first,File Path,HOM
0,0,0,PIG-001_J-17-0569_LM_HE.ndpi,0.0,0.0,NaN,1,0.0,0.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,0.0
1,86,4,PIG-001_J-17-0569_LM_HE.ndpi,0.0,0.0,NaN,1,0.0,0.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,0.0
2,87,4,PIG-001_J-17-0571_LM central_HE.ndpi,0.0,0.0,NaN,1,0.0,0.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,0.0
3,151,7,PIG-001_J-17-0569_LM_HE.ndpi,0.0,0.0,NaN,1,0.0,0.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,0.0
4,172,8,PIG-001_J-17-0571_LM central_HE.ndpi,0.0,0.0,NaN,1,0.0,0.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,0.0
...,...,...,...,...,...,...,...,...,...,...,...
243,74,3,PIG-003_J-18-0170_HE.ndpi,1.0,1.0,NaN,1,1.0,1.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,1.0
244,75,3,PIG-004_BBJ-004-2 _HE_parenchyme.ndpi,1.0,1.0,NaN,1,1.0,1.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,1.0
245,63,2,PIG-012_Pilsen_HE_LMC.ndpi,1.0,1.0,NaN,1,1.0,1.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,1.0
246,123,5,PIG-005_J-18-0633_HE_PRML per decell.ndpi,1.0,1.0,NaN,1,1.0,1.0,C:\Users\Jirik\data\medical\orig\Scaffan-analy...,1.0


### Add the column

In [8]:
df_new_1 = imst.add_parameter_column(df, df_micrant_parameter, colname)
df_new_1

2020-04-11 22:52:33.044 | DEBUG    | micrant.image_sort_tools:get_new_parameter_table:40 - No Annotation Method or Annotation Details column


,Annotation Color,Annotation Details,Annotation ID,Annotation Title,Area,Area unit,Branch number,Central Vein Dice,Central Vein Jaccard,Central vein area,...,Processing;Texture Analysis;GLCM Levels,Processing;Texture Analysis;Tile Size,Processing;Texture Analysis;Tile Spacing,Processing;Texture Analysis;Working Resolution,Scaffan Version,Skeleton length,platform.node,platform.processor,platform.system,HOM
0,#00FF00,PFCVI=2 SNI=2 SWP=1 SQZ=0.4 CVM=0 IFE=0,6,NaN,0.777046,mm,22489.0,NaN,NaN,0.056655,...,64,64,32,4.000000e-07,0.14.0,45.58554,krux2.ncbr.muni.cz,NaN,Linux,0.75
1,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.8,6,NaN,1.959020,mm,8871.0,NaN,NaN,0.085619,...,64,64,32,4.000000e-07,0.14.0,18.66592,konos2.fav.zcu.cz,NaN,Linux,0.00
2,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,1,NaN,1.562744,mm,NaN,0.108730,0.057491,0.137554,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
3,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,2,NaN,1.031514,mm,NaN,0.072651,0.037695,0.148508,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
4,#00FF00,PFCVI=1 SNI=0 SWP=0 SQZ=0.1,6,NaN,1.184912,mm,NaN,0.035911,0.018284,0.148555,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=1.5,9,NaN,0.920082,mm,14435.0,NaN,NaN,0.074718,...,64,64,32,4.000000e-07,0.14.0,28.39382,elmo5-14.hw.elixir-czech.cz,NaN,Linux,0.25
316,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,10,NaN,0.817524,mm,15528.0,NaN,NaN,0.027523,...,64,64,32,4.000000e-07,0.14.0,29.15094,elmo5-14.hw.elixir-czech.cz,NaN,Linux,0.75
317,#0000FF,IFE=0 CVM=0 SQZ=0.1 SNI=2.0,11,NaN,1.190702,mm,15148.0,NaN,NaN,0.119581,...,64,64,32,4.000000e-07,0.14.0,26.50921,elmo5-14.hw.elixir-czech.cz,NaN,Linux,1.00
318,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,12,NaN,1.728910,mm,35651.0,NaN,NaN,0.096388,...,64,64,32,4.000000e-07,0.14.0,65.94952,elmo5-14.hw.elixir-czech.cz,NaN,Linux,1.00


### Add column without MicrAnt

In [9]:
# df_sni_reconstruction = imst.get_new_parameter_table(
#     df_micrant, colname=colname, rewrite_annotated_parameter_with_recent=True
# )

df_new_2 = df.join(
    df_micrant_parameter[["Annotation ID", "File Name", colname]].set_index(
        ["Annotation ID", "File Name"]
    ),
    on=["Annotation ID", "File Name"],
)
df_new_2

,Annotation Color,Annotation Details,Annotation ID,Annotation Title,Area,Area unit,Branch number,Central Vein Dice,Central Vein Jaccard,Central vein area,...,Processing;Texture Analysis;GLCM Levels,Processing;Texture Analysis;Tile Size,Processing;Texture Analysis;Tile Spacing,Processing;Texture Analysis;Working Resolution,Scaffan Version,Skeleton length,platform.node,platform.processor,platform.system,HOM
0,#00FF00,PFCVI=2 SNI=2 SWP=1 SQZ=0.4 CVM=0 IFE=0,6,NaN,0.777046,mm,22489.0,NaN,NaN,0.056655,...,64,64,32,4.000000e-07,0.14.0,45.58554,krux2.ncbr.muni.cz,NaN,Linux,0.75
1,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.8,6,NaN,1.959020,mm,8871.0,NaN,NaN,0.085619,...,64,64,32,4.000000e-07,0.14.0,18.66592,konos2.fav.zcu.cz,NaN,Linux,0.00
2,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,1,NaN,1.562744,mm,NaN,0.108730,0.057491,0.137554,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
3,#00FF00,PFCVI=1 SNI=0 SWP=0 CMP=0.9,2,NaN,1.031514,mm,NaN,0.072651,0.037695,0.148508,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
4,#00FF00,PFCVI=1 SNI=0 SWP=0 SQZ=0.1,6,NaN,1.184912,mm,NaN,0.035911,0.018284,0.148555,...,64,64,32,4.000000e-07,0.14.0,NaN,charon01.nti.tul.cz,NaN,Linux,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=1.5,9,NaN,0.920082,mm,14435.0,NaN,NaN,0.074718,...,64,64,32,4.000000e-07,0.14.0,28.39382,elmo5-14.hw.elixir-czech.cz,NaN,Linux,0.25
316,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,10,NaN,0.817524,mm,15528.0,NaN,NaN,0.027523,...,64,64,32,4.000000e-07,0.14.0,29.15094,elmo5-14.hw.elixir-czech.cz,NaN,Linux,0.75
317,#0000FF,IFE=0 CVM=0 SQZ=0.1 SNI=2.0,11,NaN,1.190702,mm,15148.0,NaN,NaN,0.119581,...,64,64,32,4.000000e-07,0.14.0,26.50921,elmo5-14.hw.elixir-czech.cz,NaN,Linux,1.00
318,#0000FF,IFE=0 CVM=0 SQZ=0.2 SNI=2.0,12,NaN,1.728910,mm,35651.0,NaN,NaN,0.096388,...,64,64,32,4.000000e-07,0.14.0,65.94952,elmo5-14.hw.elixir-czech.cz,NaN,Linux,1.00


# Use two columns described with MicrAnt

In [13]:
df_sni = imst.get_new_parameter_table(df, colname="SNI", rewrite_annotated_parameter_with_recent=True)
df_both = imst.add_parameter_column(df_sni, df_micrant_parameter, "HOM")
# df_both
df_both[["File Name", "Annotation ID", "SNI", "HOM"]]

df_both.to_excel("parameter_table_SNI_HOM.recent.xlsx")

2020-04-11 22:55:51.091 | DEBUG    | micrant.image_sort_tools:get_new_parameter_table:37 - Creating column SNI from Annotation Details
2020-04-11 22:55:51.119 | DEBUG    | micrant.image_sort_tools:get_new_parameter_table:40 - No Annotation Method or Annotation Details column
